In [1]:
import sys

sys.path.append("../../..")

from ca_models_meta import CurrentAccounts
from pyspark.sql import SparkSession

In [6]:
CurrentAccounts.models

[ModelMetaInfo(model_name='current_accounts_model', model_trainer=<core.models.current_accounts.general_balance.current_accounts_model.CurrentAccountsModelTrainer object at 0x7fe3ed346510>, data_loader=<core.models.current_accounts.general_balance.current_accounts_model.CurrentAccountsDataLoader object at 0x7fe3ed346410>, adapter=<class 'core.models.current_accounts.general_balance.current_accounts_model.CurrentAccountsModelAdapter'>, segment=None, replenishable_flg=None, subtraction_flg=None),
 ModelMetaInfo(model_name='current_accounts_segment_structure_model', model_trainer=<core.models.current_accounts.segment_structure.segment_structure_model.CaSegmentStructureModelTrainer object at 0x7fe3ece42c50>, data_loader=<core.models.current_accounts.segment_structure.segment_structure_model.CaSegmentStructureDataLoader object at 0x7fe3ece42f10>, adapter=<class 'core.models.current_accounts.segment_structure.segment_structure_model.CaSegmentStructureModelAdapter'>, segment=None, replenishab

In [2]:
spark = (
    SparkSession.builder.master("yarn")
    .appName("overwrite_deposit_feature_tables")
    .getOrCreate()
)
spark.sparkContext.setLogLevel("ERROR")
spark.conf.set("spark.sql.session.timeZone", "GMT")
spark.conf.set("spark.sql.execution.arrow.enabled", "false")
spark.conf.set(
    "spark.sql.legacy.allowCreatingManagedTableUsingNonemptyLocation", "true"
)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/09 13:35:39 WARN util.Utils: Service 'sparkDriver' could not bind on port 31850. Attempting port 31851.
23/06/09 13:35:39 WARN util.Utils: Service 'sparkDriver' could not bind on port 31851. Attempting port 31852.
23/06/09 13:35:39 WARN util.Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/06/09 13:35:39 WARN util.Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/06/09 13:35:43 WARN yarn.Client: Same path resource file:///opt/cloudera/parcels/AnacondaPy37/jars/spark-tree-plotting-0.2.jar added multiple times to distributed cache.
23/06/09 13:36:18 WARN util.Utils: Service 'org.apache.spark.network.netty.NettyBlockTransferService' could not bind on port 63695. Attempting port 63696.
23/06/09 13:36:18 WARN util.Utils: Service 'org.apache.spark.network.netty.NettyBlockTransferService' could not bi

In [10]:
CurrentAccounts.model_name

'current_accounts_model'

In [5]:
from datetime import datetime

data = CurrentAccounts.data_loader.get_training_data(
    spark, start_date=datetime(2010, 2, 28), end_date=datetime(2023, 4, 30)
)

In [6]:
data["target"]

,balance_rur
report_dt,
2010-02-28,74.437005
2010-03-31,75.651323
2010-04-30,78.780435
2010-05-31,82.123447
2010-06-30,86.817299
...,...
2022-12-31,544.139323
2023-01-31,552.941526
2023-02-28,541.532686


In [3]:
import pandas as pd

date = "2022-04-30"

In [10]:
X, y = (
    data["features"][data["features"].index < date],
    data["target"][data["target"].index < date],
)

In [15]:
pred = data["features"][data["features"].index >= date]

In [3]:
X.to_parquet("X")
y.to_parquet("y")
pred.to_parquet("pred")

NameError: name 'X' is not defined

In [4]:
X = pd.read_parquet("X")
y = pd.read_parquet("y")
pred = pd.read_parquet("pred")

In [5]:
m = CurrentAccountsModel()

In [6]:
m.fit(X, y)

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Initial log joint probability = -2.52937
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       520.433    0.00139311       40.1026           1           1      116   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     144       520.531   3.53528e-06       2.73646    1.12e-06       0.001      198  LS failed, Hessian reset 
     170       520.532   1.01003e-05       1.82109      0.2732           1      238   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance


In [7]:
m.predict(pred)

,ds,rate_ruonia,covid_dummy,svo_dummy
0,2022-04-30,0.171267,0,1
1,2022-05-31,0.132978,0,1
2,2022-06-30,0.096329,0,1
3,2022-07-31,0.089038,0,1
4,2022-08-31,0.079470,0,1
5,2022-09-30,0.076914,0,1
6,2022-10-31,0.077171,0,1
7,2022-11-30,0.073333,0,1
8,2022-12-31,0.073190,0,1
9,2023-01-31,0.070000,0,1


,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,covid_dummy,...,svo_dummy,svo_dummy_lower,svo_dummy_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2022-04-30,492.783227,444.013068,468.364967,492.783227,492.783227,-36.960246,-36.960246,-36.960246,0.0,...,-2.048664,-2.048664,-2.048664,-3.278767,-3.278767,-3.278767,0.0,0.0,0.0,455.822981
1,2022-05-31,497.196412,461.922234,486.437554,497.196412,497.196412,-23.151112,-23.151112,-23.151112,0.0,...,-2.048664,-2.048664,-2.048664,-1.579807,-1.579807,-1.579807,0.0,0.0,0.0,474.045299
2,2022-06-30,501.467236,482.666380,508.387964,501.467236,501.467236,-5.894905,-5.894905,-5.894905,0.0,...,-2.048664,-2.048664,-2.048664,4.084829,4.084829,4.084829,0.0,0.0,0.0,495.572330
3,2022-07-31,505.880421,495.098975,520.744780,505.880421,505.880421,1.972899,1.972899,1.972899,0.0,...,-2.048664,-2.048664,-2.048664,9.646767,9.646767,9.646767,0.0,0.0,0.0,507.853319
4,2022-08-31,510.293605,496.573636,520.697514,510.293605,510.293605,-1.532036,-1.532036,-1.532036,0.0,...,-2.048664,-2.048664,-2.048664,3.115455,3.115455,3.115455,0.0,0.0,0.0,508.761570
5,2022-09-30,514.564429,493.157827,517.321006,514.564429,514.564429,-9.335344,-9.335344,-9.335344,0.0,...,-2.048664,-2.048664,-2.048664,-5.496253,-5.496253,-5.496253,0.0,0.0,0.0,505.229086
6,2022-10-31,518.977614,493.155543,518.332224,518.977614,518.977614,-13.583825,-13.583825,-13.583825,0.0,...,-2.048664,-2.048664,-2.048664,-9.663198,-9.663198,-9.663198,0.0,0.0,0.0,505.393789
7,2022-11-30,523.248438,495.239851,519.773989,523.248438,523.248438,-15.743477,-15.743477,-15.743477,0.0,...,-2.048664,-2.048664,-2.048664,-13.036781,-13.036781,-13.036781,0.0,0.0,0.0,507.504961
8,2022-12-31,527.661623,520.120375,543.550983,527.661623,527.661623,4.407135,4.407135,4.407135,0.0,...,-2.048664,-2.048664,-2.048664,7.068650,7.068650,7.068650,0.0,0.0,0.0,532.068758
9,2023-01-31,532.074808,520.707097,545.511469,531.987183,532.074808,1.747233,1.747233,1.747233,0.0,...,-2.048664,-2.048664,-2.048664,3.399649,3.399649,3.399649,0.0,0.0,0.0,533.822041


,balance_rur
report_dt,
2022-04-30,455.822981
2022-05-31,474.045299
2022-06-30,495.572330
2022-07-31,507.853319
2022-08-31,508.761570
2022-09-30,505.229086
2022-10-31,505.393789
2022-11-30,507.504961
2022-12-31,532.068758


In [7]:
pd.to_datetime(date)

Timestamp('2022-04-30 00:00:00')